In [2]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


## Yahoo Fundamental

In [ ]:
def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
    """
    Extrat data from trading economics callendar.

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if table_class:
            tabela = soup.find('table', {'class': table_class})
        else:
            tabela = soup.find('table')
        
        headers = [th.text.strip() for th in tabela.find_all('th')]
        print(headers)
        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        

    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

    return df


In [ ]:
def get_whalewisdom(ticker: str) -> pd.DataFrame:
    """
    Extracts data from WhaleWisdom for a given stock ticker.

    Parameters:
        ticker (str): Stock ticker symbol.

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontrar a div com a classe "v-data-table__wrapper"
        div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
        if not div_wrapper:
            raise ValueError("Div com a classe 'v-window__container' não encontrada.")

        # Buscar a primeira tabela dentro desta div
        tabela = div_wrapper.find('table')
        if not tabela:
            raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
        # Coletar os cabeçalhos da tabela ignorando ícones
        headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
        # print(headers)

        # Encontrando todas as linhas dentro da tabela (tbody > tr)
        rows = tabela.find("tbody").find_all("tr")
        print(rows)
        # Extraindo os dados de cada linha
        data = []
        for row in rows:
            columns = row.find_all("td")

            # Pegando os valores correspondentes
            institution_name = columns[0].text.strip()
            shares_held = columns[2].text.strip()
            portfolio_value = columns[3].text.strip()
            percentage_ownership = columns[4].text.strip()
            last_report_date = columns[-1].text.strip()

            data.append({
                "Instituição": institution_name,
                "Ações Detidas": shares_held,
                "Valor do Portfólio": portfolio_value,
                "Percentual de Participação": percentage_ownership,
                "Última Atualização": last_report_date
            })
        #print(data)
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except ValueError as e:
        print(f"Erro na extração de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")
    
    return pd.DataFrame()

In [52]:
df_whale_s = get_whalewisdom(ticker = "ctre")
df_whale_s

[<tr class="v-data-table__empty-wrapper"><td colspan="15">No data available</td></tr>]
Erro inesperado: list index out of range


""


In [67]:
base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json"
}

response = requests.get(base_url, headers=headers)
response.raise_for_status()

response.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Track Hedge Funds Using 13F Filings </title>\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">\n            <meta name="description" content="WhaleWisdom tracks 13F,  Schedule 13D, and 13G EDGAR filings by hedge funds. Hedge Fund Whale Backtesting and search tools "/>\n    <link rel="shortcut icon" type="image/x-icon" href="https://d27mjrcvcy56qq.cloudfront.net/images/favicon.ico"/>\n    <link rel="apple-touch-icon" sizes="57x57" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-57x57.png">\n    <link rel="apple-touch-icon" sizes="60x60" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-60x60.png">\n    <link rel="apple-touch-icon" sizes="72x72" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-72x72.png">\n    <link rel="apple-touch-icon" sizes="76x76" href="https://d27mjrcvcy56

In [25]:
import hashlib
import hmac
import base64
import json
import time
import urllib.parse
import requests

class WhaleWisdom:
    def __init__(self, secret_key, shared_key):
        self.secret_key = secret_key
        self.shared_key = shared_key
        self.timestamp = self.get_timestamp()

    def get_timestamp(self):
        """Gera o timestamp no formato UTC"""
        return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

    def signature(self, args):
        """Gera a assinatura HMAC-SHA1 em Base64"""
        message = f"{args}\n{self.timestamp}".encode('utf-8')
        hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
        return base64.b64encode(hmac_digest).decode('utf-8')

    def encode(self, string):
        """Codifica a string para URL"""
        return urllib.parse.quote(string, safe='')

    def endpoint(self, args):
        """Constrói a URL da API do WhaleWisdom"""
        encoded_args = self.encode(json.dumps(args))
        api_sig = self.signature(json.dumps(args))
        encoded_timestamp = self.encode(self.timestamp)

        return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

    def request(self, args):
        """Faz a requisição à API"""
        url = self.endpoint(args)
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(f"Erro {response.status_code}: {response.text}")
            return None


# **Exemplo de Uso**
WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# Obter as participações (holdings) de um gestor específico
response = ww.request({
    "command": "stock_lookup",
    "symbol": "ctre"
})

data = json.loads(response)

stock_id = data["stocks"][0]["id"]
name = data["stocks"][0]["name"]
status = data["stocks"][0]["status"]
link = data["stocks"][0]["link"]

stock_id

167941

In [36]:
response = ww.request(
{"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
response


'{"errors":["Subscription limit has been reached for current period"]}'

In [19]:
response

'{"errors":["Subscription limit has been reached for current period"]}'

## STOCKS



In [1]:
import yfinance as yf
import pandas as pd
import talib
from django.http import JsonResponse
from datetime import datetime, timedelta

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker(symbol).info

In [3]:
def get_sector_etf_info(sector_name, search_value: str = "info" ):
    """
    Return information about symbol ETF sector.
    """
    sector_map = {
        "Technology": "XLK",
        "Financial Services": "XLF",
        "Consumer Cyclical": "XLY",
        "Healthcare": "XLV",
        "Communication Services": "XLC",
        "Industrials": "XLI",
        "Consumer Defensive": "XLP",
        "Energy": "XLE",
        "Real Estate": "XLRE",
        "Basic Materials": "XLB",
        "Utilities": "XLU",
    }

    etf_symbol = sector_map.get(sector_name)
    if not etf_symbol:
        return "N/A"

    ticker = yf.Ticker(etf_symbol)

    if search_value == "info":
        return ticker.info
    else:
        return ticker.info.get(search_value, "N/A")
        

# Exemplo de uso
sector = "Utilities"
info = get_sector_etf_info(sector, "info")
print(f"Informações do setor {sector}:")
print(info)

Informações do setor Utilities:
{'longBusinessSummary': 'In seeking to track the performance of the index, the fund employs a replication strategy. It generally invests substantially all, but at least 95%, of its total assets in the securities comprising the index. The index includes securities of companies from the following industries: electric utilities; water utilities; multi-utilities; independent power and renewable electricity producers; and gas utilities. The fund is non-diversified.', 'companyOfficers': [], 'executiveTeam': [], 'maxAge': 86400, 'priceHint': 2, 'previousClose': 77.36, 'open': 77.59, 'dayLow': 77.31, 'dayHigh': 78.49, 'regularMarketPreviousClose': 77.36, 'regularMarketOpen': 77.59, 'regularMarketDayLow': 77.31, 'regularMarketDayHigh': 78.49, 'trailingPE': 20.296629, 'volume': 4299379, 'regularMarketVolume': 4299379, 'averageVolume': 10192532, 'averageVolume10days': 11857720, 'averageDailyVolume10Day': 11857720, 'bid': 78.3, 'ask': 78.31, 'bidSize': 14, 'askSize'

In [5]:
sector_info = yf.Ticker("XLE").info
sector_info

{'longBusinessSummary': 'In seeking to track the performance of the index, the fund employs a replication strategy. It generally invests substantially all, but at least 95%, of its total assets in the securities comprising the index. The index includes companies that have been identified as Energy companies by the GICS®, including securities of companies from the following industries: oil, gas and consumable fuels; and energy equipment and services. It is non-diversified.',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 87.18,
 'open': 87.66,
 'dayLow': 87.265,
 'dayHigh': 89.635,
 'regularMarketPreviousClose': 87.18,
 'regularMarketOpen': 87.66,
 'regularMarketDayLow': 87.265,
 'regularMarketDayHigh': 89.635,
 'trailingPE': 15.523268,
 'volume': 7074389,
 'regularMarketVolume': 7074404,
 'averageVolume': 16137232,
 'averageVolume10days': 20426350,
 'averageDailyVolume10Day': 20426350,
 'bid': 89.23,
 'ask': 89.25,
 'bidSize': 30,
 'a

In [71]:
yahoo_symbol_info_fast = yf.Ticker("XLT").info
yahoo_symbol_info_fast

{'quoteType': 'ECNQUOTE',
 'symbol': 'XLT',
 'language': 'en-US',
 'region': 'US',
 'typeDisp': 'ECNQUOTE',
 'quoteSourceName': 'Delayed Quote',
 'triggerable': True,
 'customPriceAlertConfidence': 'HIGH',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'gmtOffSetMilliseconds': -14400000,
 'market': 'us_market',
 'esgPopulated': False,
 'marketState': 'POST',
 'sourceInterval': 15,
 'exchangeDataDelayedBy': 0,
 'tradeable': False,
 'cryptoTradeable': False,
 'priceHint': 2,
 'fullExchangeName': 'NasdaqGS',
 'corporateActions': [],
 'exchange': 'NMS',
 'hasPrePostMarketData': False,
 'trailingPegRatio': None}

In [64]:
yahoo_symbol_info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [22]:
category = yahoo_tec_etf_info.get("category", "N/A")
category

'Technology'

In [24]:
sector == category

True

In [ ]:
yahoo_symbol_info = yf.Ticker(symbol).info
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_income = yf.Ticker(symbol).income_stmt

In [4]:
yahoo_symbol_income.loc["EBIT"].iloc[0]

123216000000.0

In [ ]:
# Market Price vs Value 

In [7]:
yahoo_symbol_info

{'longBusinessSummary': 'In seeking to track the performance of the index, the fund employs a replication strategy, which means that the fund typically invests in substantially all of the securities represented in the index in approximately the same proportions as the index. It generally invests substantially all, but at least 95%, of its total assets in the securities comprising the index. The fund is non-diversified.',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 211.68,
 'open': 211.07,
 'dayLow': 206.8475,
 'dayHigh': 211.54,
 'regularMarketPreviousClose': 211.68,
 'regularMarketOpen': 211.07,
 'regularMarketDayLow': 206.8475,
 'regularMarketDayHigh': 211.54,
 'trailingPE': 32.97669,
 'volume': 6183607,
 'regularMarketVolume': 6183607,
 'averageVolume': 4943936,
 'averageVolume10days': 7075400,
 'averageDailyVolume10Day': 7075400,
 'bid': 207.8,
 'ask': 208.79,
 'bidSize': 8,
 'askSize': 8,
 'yield': 0.0068,
 'totalAssets': 7091

In [8]:
# yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
# # yahoo_symbol_balancesheet.index

In [9]:
# yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
# yahoo_symbol_cashflow.index

In [10]:
# revenue_history = yahoo_symbol_info.get("netIncomeToCommon")
# revenue_history

In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
import requests

url = "https://api.binance.com/api/v3/ticker/price"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)

if response.status_code == 200:
    symbol = response.json()["symbol"]
    price = response.json()["price"]
else:
    error = response.status_code

In [ ]:
from decimal import Decimal
from datetime import datetime


def get_crypto_symbol_24h(symbol : str):

    url = "https://api.binance.com/api/v3/ticker/24hr"
    params = {"symbol": symbol}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        priceChangePercent = Decimal(response.json()["priceChangePercent"])
        weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
        prevClosePrice = Decimal(response.json()["prevClosePrice"])
        priceChange = Decimal(response.json()["priceChange"])
        lastPrice = Decimal(response.json()["lastPrice"])
        lastQty = Decimal(response.json()["lastQty"])
        bidPrice = Decimal(response.json()["bidPrice"])
        bidQty = Decimal(response.json()["bidQty"])
        askPrice = Decimal(response.json()["askPrice"])
        askQty = Decimal(response.json()["askQty"])
        openPrice = Decimal(response.json()["openPrice"])
        highPrice = Decimal(response.json()["highPrice"])
        lowPrice = Decimal(response.json()["lowPrice"])
        volume = Decimal(response.json()["volume"])
        quoteVolume = Decimal(response.json()["quoteVolume"])
        openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        firstId = response.json()["firstId"]
        lastId = response.json()["lastId"]
        count = response.json()["count"]
    else:
        print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
url = "https://api.binance.com/api/v3/openOrders"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)
response.json()


## TEST

In [1]:
def detect_pattern(data, pattern_function, pattern_name: str, dates):
    """
    General method to detect a specific candlestick pattern.
    """

    detection = pattern_function(data['Open'], data['High'], data['Low'], data['Close'])
    detected_indices = np.nonzero(detection)[0]

    results = []
    three_months_ago = datetime.now() - timedelta(days=90)

    for i in detected_indices:
        date = dates[i]

        # Ignorar datas antigas irrelevantes
        if datetime.strptime(date, "%Y-%m-%d %H:%M") < three_months_ago:
            continue  

        signal = int(detection[i])

        # Determinar stop-loss com base no tipo de padrão
        if pattern_name in [
            "doji", "dragonfly_doji", "gravestone_doji", "engulfing",
            "morning_star", "evening_star", "marubozu", "harami",
            "harami_cross", "kicking", "kicking_by_length", "tasuki_gap",
            "gap_side_by_side_white", "counter_attack", "piercing",
            "dark_cloud_cover", "tri_star", "spinning_top"
        ]:
            stoploss = round(data['Low'][i], 5) if signal > 0 else round(data['High'][i], 5)
        elif pattern_name in [
            "morning_doji_star", "hammer", "inverted_hammer",
            "thrusting", "matching_low", "three_white_soldiers",
            "three_outside", "three_stars_in_south"
        ]:
            stoploss = round(data['Low'][i], 5)
        elif pattern_name in [
            "evening_doji_star", "hanging_man", "shooting_star",
            "on_neck", "in_neck", "three_black_crows",
            "three_inside", "advance_block", "stalled_pattern"
        ]:
            stoploss = round(data['High'][i], 5)
        else:
            stoploss = None

        # Verificação se o Stoploss foi atingido
        hit_stoploss = "N/A"

        if stoploss is not None:
            future_close_prices = data['Close'][i + 1:]

            if signal == -100:
                hit_stoploss = "Hit Stoploss (Above)" if any(close > stoploss for close in future_close_prices) else "No Hit"

            elif signal == 100:
                hit_stoploss = "Hit Stoploss (Below)" if any(close < stoploss for close in future_close_prices) else "No Hit"

        # Adicionar ao resultado apenas se houver valores válidos
        if stoploss:
            results.append({
                'Signal': signal,
                'Stoploss': stoploss,
                'Date': date,
                'Result': hit_stoploss
            })

    return results


In [2]:
def doji(self, data, dates): return self.detect_pattern(data, talib.CDLDOJI, "doji", dates)
def dragonfly_doji(self, data, dates): return self.detect_pattern(data, talib.CDLDRAGONFLYDOJI, "dragonfly_doji", dates)
def gravestone_doji(self, data, dates): return self.detect_pattern(data, talib.CDLGRAVESTONEDOJI, "gravestone_doji", dates)
def engulfing(self, data, dates): return self.detect_pattern(data, talib.CDLENGULFING, "engulfing", dates)
def morning_star(self, data, dates): return self.detect_pattern(data, talib.CDLMORNINGSTAR, "morning_star", dates)
def evening_star(self, data, dates): return self.detect_pattern(data, talib.CDLEVENINGSTAR, "evening_star", dates)
def morning_doji_star(self, data, dates): return self.detect_pattern(data, talib.CDLMORNINGDOJISTAR, "morning_doji_star", dates)
def evening_doji_star(self, data, dates): return self.detect_pattern(data, talib.CDLEVENINGDOJISTAR, "evening_doji_star", dates)
def hammer(self, data, dates): return self.detect_pattern(data, talib.CDLHAMMER, "hammer", dates)
def inverted_hammer(self, data, dates): return self.detect_pattern(data, talib.CDLINVERTEDHAMMER, "inverted_hammer", dates)
def hanging_man(self, data, dates): return self.detect_pattern(data, talib.CDLHANGINGMAN, "hanging_man", dates)
def shooting_star(self, data, dates): return self.detect_pattern(data, talib.CDLSHOOTINGSTAR, "shooting_star", dates)
def marubozu(self, data, dates): return self.detect_pattern(data, talib.CDLMARUBOZU, "marubozu", dates)
def harami(self, data, dates): return self.detect_pattern(data, talib.CDLHARAMI, "harami", dates)
def harami_cross(self, data, dates): return self.detect_pattern(data, talib.CDLHARAMICROSS, "harami_cross", dates)
def spinning_top(self, data, dates): return self.detect_pattern(data, talib.CDLSPINNINGTOP, "spinning_top", dates)
def kicking(self, data, dates): return self.detect_pattern(data, talib.CDLKICKING, "kicking", dates)
def kicking_by_length(self, data, dates): return self.detect_pattern(data, talib.CDLKICKINGBYLENGTH, "kicking_by_length", dates)
def tasuki_gap(self, data, dates): return self.detect_pattern(data, talib.CDLTASUKIGAP, "tasuki_gap", dates)
def gap_side_by_side_white(self, data, dates): return self.detect_pattern(data, talib.CDLGAPSIDESIDEWHITE, "gap_side_by_side_white", dates)
def counter_attack(self, data, dates): return self.detect_pattern(data, talib.CDLCOUNTERATTACK, "counter_attack", dates)
def piercing(self, data, dates): return self.detect_pattern(data, talib.CDLPIERCING, "piercing", dates)
def dark_cloud_cover(self, data, dates): return self.detect_pattern(data, talib.CDLDARKCLOUDCOVER, "dark_cloud_cover", dates)
def tri_star(self, data, dates): return self.detect_pattern(data, talib.CDLTRISTAR, "tri_star", dates)
def on_neck(self, data, dates): return self.detect_pattern(data, talib.CDLONNECK, "on_neck", dates)
def in_neck(self, data, dates): return self.detect_pattern(data, talib.CDLINNECK, "in_neck", dates)
def thrusting(self, data, dates): return self.detect_pattern(data, talib.CDLTHRUSTING, "thrusting", dates)
def matching_low(self, data, dates): return self.detect_pattern(data, talib.CDLMATCHINGLOW, "matching_low", dates)
def three_black_crows(self, data, dates): return self.detect_pattern(data, talib.CDL3BLACKCROWS, "three_black_crows", dates)
def three_white_soldiers(self, data, dates): return self.detect_pattern(data, talib.CDL3WHITESOLDIERS, "three_white_soldiers", dates)
def three_inside(self, data, dates): return self.detect_pattern(data, talib.CDL3INSIDE, "three_inside", dates)
def three_outside(self, data, dates): return self.detect_pattern(data, talib.CDL3OUTSIDE, "three_outside", dates)
def three_stars_in_south(self, data, dates): return self.detect_pattern(data, talib.CDL3STARSINSOUTH, "three_stars_in_south", dates)
def advance_block(self, data, dates): return self.detect_pattern(data, talib.CDLADVANCEBLOCK, "advance_block", dates)
def stalled_pattern(self, data, dates): return self.detect_pattern(data, talib.CDLSTALLEDPATTERN, "stalled_pattern", dates)